# Metaheuristic Algorithms

##  Index
1. Problem
1. Review and Implementation Algorithms
    1. GA
    1. DE 
    1. PSO
1. Evaluation
    1. Performance
        1. Accuracy
        1. Time
        1. Memory

## 1. Probrem
Try to find the minimum points of the following functions, using GA, DE, and PSO and compare their performance

$$f_1(x_1,x_2,x_3)=\sum_{i=1}^3 x_i^2, 5.11 \leq x_i \leq 5.12$$

$$f_2(x_1,x_2)=100(x_1^2 - x_2)^2 + (1-x_1)^2, -2.047 \leq x_i  \leq 2.048 $$

![3Dgraph](report_image/1.png)
![3Dgraph](report_image/2.png)

$F1 min : 0.0 \; (0.01039) \; \; \; F2 min : 0.0 \;(0.07062$)


### 2. Review and Implementation algorithms (keypoint)
- ####  Genetic Algorithm (GA)
Selection, crossover and mutation
![FC_GA](report_image/FC_GA.png)
> cite: Handout Lec06


#### Reconstruction of the phenotype from the genotype site

>$y_{i}=\sigma_{k=1}^{B}b_{ik}2^{-k},    \;\; w_{i}=a[y_{i}]+b$

> where $a$ and $b$ are the scaling and shifting factors.

>cite: Handout Lec06 p.22



In [1]:
class Gene:    
    #-------------------------------------------------------
    # This class has a genotype(M bits binary) and a phenotype (decimal number collesponding the genotype)
    #
    # obj: objective function
    # M : the length of genotype
    # dim : the dimension of the solution space 
    # a : scaling
    # b : shifting
    #-------------------------------------------------------
    
    def __init__(self, obj, M,dim,a,b, genotype=None):
        if genotype is None:
            # generate M-bits binary dim times
            self.genotype = [[int(g[k]%2) for k in range(0,M) for g in [random(M)*10]] for x in range(0,dim)]
            # decode the genotype
            self.phenotype = [sum([g[i]*2**(-i) for i in range(0,len(g))])*a-b for g in self.genotype]
            self.fitness = obj(self.phenotype)
        else:
            self.genotype = genotype
            self.phenotype = [sum([g[i]*2**(-i) for i in range(0,len(g))])*a-b for g in self.genotype]
            self.fitness = obj(self.phenotype)   
    
    def getGenotype(self):
        return self.genotype

    def getPhenotype(self):
        return self.phenotype   
    
    def setFitness(self, phenotype):
        self.fitness = obj(self.phenotype)

    def setPhenotype(self,genotype):
        self.phenotype = [sum([g[i]*2**(-i) for i in range(0,len(g))])*a-b for g in genotype]
        self.setFitness(self.phenotype)
        
    def setGenotype(self,genotype):
        self.genotype = genotype
        self.setPhenotype(self.genotype)
     

In [2]:
def GA(obj,dim) :
    N = 10 # the number of population
    M = 10 # the lenght of genotype
    sr = 0.7 # selection rate
    a = 2.5
    b = 0
    popSize = int(N*sr) # the number of trucation selection (correspond to selection rate)
    pm = 0.6
    
    def crossover(chromosomes):  
        crossover_point = randint(0,len(chromosomes[0]))    
        temp = chromosomes[0][crossover_point:]
        chromosomes[0][crossover_point:] = chromosomes[1][crossover_point:]
        chromosomes[1][crossover_point:] = temp
        return chromosomes

    def mutation(gene,mutation_rate):
        dim = len(gene.genotype)
        def generate_mutation(genotype,pm):
            return [(bit+int(random()>pm))%2 for bit in genotype]    
        gene.setGenotype([generate_mutation(gene.genotype[i],mutation_rate) for i in range(0,dim)])
    
    #Step 1 Initialization
    population = [Gene(obj,M,dim,a,b) for g in range(0,N)] # Step 2 Evaluation


    for cnt in range(0,300):
        # Step 3.1 Selection (by tounament)
        new_population = sorted(population, key=lambda population: population.fitness) # sorted by fitness
        [new_population.pop(-1) for i in range(0,popSize)] # trunction selection  
       
        # Step 3.2 Reproduction      
        # for each new population
        for i in range(N-popSize,len(population)):
            # select parents randomly             
            parents = [new_population[randint(0,N-popSize)] for i in range(0,2)]
            # produce children genotype by crossover the parents
            children_geno = [crossover([parent.genotype[i] for i in range(0,dim)]) for parent in parents][randint(0,2)]
            new_population.append(Gene(obj,M,dim,a,b,children_geno))     
        [mutation(p,pm) for p in population]
        population = new_population
        

#### F1
![GA_objhistory_f1](report_image/GA_objhistory_f1.png)
![GA_objhistory_f1](report_image/GA_cnt_history_f1.png)
#### F2
![GA_objhistory_f2](report_image/GA_objhistory_f2.png)
![GA_objhistory_f2](report_image/GA_cnt_history_f2.png)



#### Differential Evolution (DE)
Perubtation is caused by 3 agents $r_{1},r_{2},r_{3}$.


$u_i = r_{3i} + F(r_{1i} - r_{2_i}), \;\; i=1,2,...,D$
![FC_GA](report_image/FC_DE.png)
> cite: Handout Lec08

In [3]:
class Agent:
    def __init__(self,obj,vector):
        self.vector = vector
        self.fitness = obj(vector)
    
    def getVector(self):
        return self.vector

    def setVector(self, vector):
        self.vector = vector

In [4]:
def DE(obj,dim) :
    N = 10
    r = 5.11
    w = 1 # differential weight [0,2]
    CR = 0.6 # crossover rate [0.4,1]
    val_his = []
    pos_his = []
    
    def new_agent(obj,agent, population, weight, crossover_rate):
        N = len(population)
        dim = len(agent.vector)      
    
        i_1 = randint(0,N) # FIX: durty code
        i_2 = randint(0,N)
        i_3 = randint(0,N)
    
        while ((i_2 == i_3) or (i_1 == i_2) or (population[i_1] == agent)):
            i_3 = randint(0,N)
            while ((i_1 == i_2) or (population[i_1] == agent)):
                i_2 = randint(0,N)
                while (population[i_1] == agent):
                    i_1 = randint(0,N)
    
        new_vec = [population[i_3].vector[i] + weight*(population[i_1].vector[i]-population[i_2].vector[i]) for i in range(0,dim)]
        new_vec = [new_vec[i] if (random() < crossover_rate) else agent.vector[i]  for i in range(0,dim)]
        new_p = Agent(obj,new_vec)
        return new_p
    
    
    # Step 1. Initialization
    population= [Agent(obj,RV4LimitSpace(dim,r)) for i in range(0,N)]
    
    # Step 2. Get an agent
    # obtain the best fitness agent by sorting and get an head element.
    best_agent = sorted(population, key=lambda population: population.fitness)[0]

    # update information (not essential part)
    val = best_agent.fitness
    val_his.append(val)
    solution = best_agent.vector
    pos_his.append(solution)
        
    for cnt in range(0,300):
        new_population = []

        # Step 3. Perturbation
        candidates = [new_agent(obj,population[0],population, w,CR) for p in population]
   
        # Step 4. Selection
        # select an agent which has higher fitness 
        new_population = [candidates[i] if (population[i].fitness >candidates[i].fitness) else population[i] for i in range(0,N)]       
        best_agent = sorted(new_population, key=lambda new_population: new_population.fitness)[0]
        # update information (not essential part)
        val = best_agent.fitness
        val_his.append(val)
        solution = best_agent.vector
        pos_his.append(solution)
        
        population = new_population
        
    return solution, val, val_his, pos_his
    

#### F1
![DE_objhistory_f1](report_image/DE_objhistory_f1.png)
![DE_cnt_history_f1.png](report_image/DE_cnt_history_f1.png)
#### F2
![DE_objhistory_f2](report_image/DE_objhistory_f2.png)
![DE_cnt_history_f2.png](report_image/DE_cnt_history_f2.png)

#### Particle Swarm Optimization (PSO)
$v_{i}(k+1) = av_i(k)+bw_1 \times (p_{b}(k)-x(k))+ cw_2 \times (p_g(k)-x(k)) $

$k$ is itaration number
$a$ is inertia weight
$b,c$ are learning factors
$w$ is radom number

![FC_GA](report_image/FC_PSO.png)
> cite: Handout Lec09

In [5]:
class Particle:
    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity
        self.fitness = 0.0
    
    def getPosition(self):
        return self.position

    def getVelocity(self):
        return self.velocity

    def setPosition(self, position):
        self.position = position

    def setVelocity(self,velocity):
        self.velocity = velocity

In [6]:
def PSO(obj,dim):

    def UpdateParticles(swarm, p_best, g_best):
        a = 0.01 # inertia weight
        b = 0.01 # the learning factor (personal influence)
        c = 0.01 # the learning factor (social finfluence)        

        positions = [s.position for s in swarm]    
        new_velocities = [a*s.velocity + b*random()*(p_best - s.position) + c*random()*(g_best - s.position)  for s in swarm]
        new_positions = [ s.position + v  for (s,v) in zip(swarm, new_velocities)]
        new_swarm = [Particle(new_positions[i], new_velocities[i]) for i in range(0,len(swarm))]
        return new_swarm

    swarm = []  # empty set
    N = 50 # The number of swarm
    g_best = []
    
    g_history = []
    pos_history = []

    # Set the space of finding
    if obj == f1:
        r = 5.11
    elif obj == f2:
        r = 2.048
    else : 
        r = 10
    
    # Step 1: Randomly initialize te swarm
    swarm = [Particle(RV4LimitSpace(dim,r) ,random(dim)*10) for i in range(0,N)]

    # Step 2: Evaluate all particles
    fit_val = [f1(s.position) for s in swarm]
    p_best =  swarm[fit_val.index(min(fit_val))].position 
    g_best = p_best
    
    cnt = 0
    # Step 3: For each particles
    while(obj(g_best)>1e-3):
        cnt += 1
        if cnt > 500 :
            print('Could not find')
            break
            
        # Update particles
        swarm = UpdateParticles(swarm, p_best, g_best)      
        
        # Evaluate the particle
        fit_val = [f1(s.position) for s in swarm]        
        p_best = swarm[fit_val.index(min(fit_val))].position
        
    # Step 4: Update if necessary the leader of the swarm and the best position obtained by each particles        
        if (obj(g_best) < obj(p_best)):
            g_best = g_best
        else :
            g_best = p_best
        
        g_history.append(obj(g_best))
        pos_history.append(g_best)
    # Step 5: Stop if terminating condition satisfied; return to Step 3 otherwise          
    return g_best, obj(g_best), g_history, pos_history    

#### F1
![PSO_objhistory_f1](report_image/PSO_objhistory_f1.png)
![PSO_cnt_history_f1.png](report_image/PSO_cnt_history_f1.png)
#### F2
![PSO_objhistory_f2](report_image/PSO_objhistory_f2.png)
![PSO_cnt_history_f2.png](report_image/PSO_cnt_history_f2.png)

## 3. Evaluation

### The common condition
- the number of trials : 50
- criterion: iterate 300 times
- population size : 50

- #### GA setting
    - M = 10  (the lenght of genotype)
    - sr = 0.7 (selection rate)
    - pm = 0.6 (mutation rate)

- #### DE setting
    - w = 1 (differential weight [0,2])
    - CR = 0.6 (crossover rate [0.4,1])

- #### PSO setting
    - a = 0.01  (inertia weight)
    - b = 0.01  (the learning factor (personal influence) ) 
    - c = 0.01  (the learning factor (social finfluence)  )

#### Accuracy

##### F1 
|Algorithm|Max.|Min.|Ave.|
|:-:|:-:|:-:|:-:|
|GA|0.12|0.00|0.03|
|DE|0.09|0.00|0.00|
|PSO|0.03|0.00|0.02|

##### F2
|Algorithm|Max.|Min.|Ave.|
|:-:|:-:|:-:|:-:|
|GA|0.17|0.00|0.02|
|DE|0.33|0.00|0.03|
|PSO|10.20|0.07|1.73|

#### Time [s]
##### F1
|Algorithm|Max.|Min.|Ave.|
|:-:|:-:|:-:|:-:|
|GA|2.12|1.5|1.66|
|DE|0.40|0.32|0.34|
|PSO|0.44|0.22|0.28|

##### F2
|Algorithm|Max.|Min.|Ave.|
|:-:|:-:|:-:|:-:|
|GA|2.12|1.13|1.37|
|DE|0.11|0.06|0.07|
|PSO|0.29|0.15|0.19|

#### Memory [MiB]
memory_profiler is provided officialy and BSD license.

##### F1
|Algorithm|Max.|Min.|Ave.|
|:-:|:-:|:-:|:-:|
|GA|20.12|19.75|19.87|
|DE|19.82|19.51|19.64|
|PSO|19.83|19.54|19.65|

##### F2
|Algorithm|Max.|Min.|Ave.|
|:-:|:-:|:-:|:-:|
|GA|19.98|19.66|19.78|
|DE|19.88|19.62|19.76|
|PSO|20.09|19.96|19.98|

#### Accuracy
- GA and DE obtain the global minimum stably.
- PSO could not obtain the global minimum sometimes.
    - The reason is too few iterations. I checked this algorithm also obtain the global minimum if the iteration number is big enough.

#### Time
- DE and PSO are the fastest algorithm.
- GA has bit-by-bit processing for genotype operation

#### Memory
There are no big difference about used memory in my implementation.
